In [1]:
import torch
import json
import h5py

In [2]:
feats = torch.load('/mnt/ialabnas/datasets/CLEVR_CoGenT_v1.0/images/valA-vit.pt')

In [3]:
feats.keys()

dict_keys(['image_name_to_idx', 'image_features'])

In [4]:
feats['image_features'].shape

torch.Size([15000, 197, 768])

In [5]:
import os
from pathlib import Path
from tqdm.auto import tqdm

# from model import ViTEmbedding
from PIL import Image
import torch
from torchvision import transforms
from torch.utils.data import DataLoader

class ImageDataset:
    def __init__(self, images_dir, split):
        self.images_dir = images_dir
        self.image_list = list(Path(images_dir).glob('*.png'))
        self.image_list = sorted(self.image_list,
                                 key=lambda x: int(x.name.replace('.png','').replace(f'CLEVR_{split}_','')))
        self.transforms = transforms.ToTensor()

    def __getitem__(self, idx):
        image_path = self.image_list[idx]
        image_tensor = self.transforms(self.load_image(image_path))
        return image_tensor

    def load_image(self, image_path):
        return Image.open(image_path).convert('RGB')

    def __len__(self):
        return len(self.image_list)

In [6]:
base_dir = '/workspace1/fidelrio/CLEVR_CoGenT_v1.0/'
base_dir2 = '/mnt/ialabnas/datasets/CLEVR_CoGenT_v1.0/'
    
device = torch.device('cuda')

In [7]:
# split = 'trainA'
# base_dir = '/workspace1/fidelrio/CLEVR_CoGenT_v1.0/'
# featues_path = f'{base_dir}/images/{split}-vit.pt'

# features_data = torch.load(featues_path)
# features = features_data['image_features']
# image_name_to_idx = features_data['image_name_to_idx']

# features = features.numpy()

In [8]:
# from models.load_pretrained_models import load_model
# from torchvision.models.feature_extraction import create_feature_extractor
from model import ViTEmbedding

# model = load_model('resnet50_trained_on_SIN_and_IN_then_finetuned_on_IN').to(device)

# feature_maps = []  # This will be a list of Tensors, each representing a feature map

# def hook_feat_map(mod, inp, out):
#     feature_maps.append(out.flatten(2).transpose(1,2).detach().cpu())

# model.module.layer4.register_forward_hook(hook_feat_map)


model = ViTEmbedding().to(device)

Some weights of the model checkpoint at google/vit-base-patch16-224 were not used when initializing ViTModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
split = 'valA'
images_dir = f'{base_dir}/images/{split}'
dataset = ImageDataset(images_dir, split)

# dataset.transforms = transforms.Compose([
#     transforms.Resize(256),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                          std=[0.229, 0.224, 0.225])
# ]) 

In [ ]:
for split in ['trainA', 'valA', 'valB', 'testA', 'testB']:
    images_dir = f'{base_dir}/images/{split}'
    dataset = ImageDataset(images_dir, split)

    num_workers = int(os.environ.get("SLURM_CPUS_PER_TASK", 4))
    loader = DataLoader(dataset, batch_size=32, num_workers=num_workers, pin_memory=True)

    all_outs = []
    for images in tqdm(loader):
        images = images.to(device)
        all_outs.append(model(images).detach().cpu())

    features = torch.cat(all_outs)

#     features = torch.cat(feature_maps)
#     feature_maps.clear()
    features = features.numpy()
    image_name_to_idx = {image_path.name: idx for idx, image_path in enumerate(dataset.image_list)}
    
    with h5py.File(f'{base_dir2}/images/{split}-vit.h5', 'w') as ds:
        ds.create_dataset("image_features", data=features)
        ds["image_features"].attrs["image_name_to_idx"] = json.dumps(image_name_to_idx)

  0%|          | 0/2188 [00:01<?, ?it/s]